In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import create_engine
from config import password
import numpy as np
import datetime

In [2]:
#read downloaded.csv files
file = "aqi_full_data.csv"
air_quality_df = pd.read_csv(file, index_col=0)

file2 = "ca_sites.csv"
sites_df = pd.read_csv(file2, index_col=0)

file3 = "census_info.csv"
census_df = pd.read_csv(file3, index_col=0)

In [3]:
# Historical Air Quality SQL table prep
air_quality_df = air_quality_df.reset_index().rename(columns={'State Name':'State_Name', \
                                'county Name':'County', \
                                'Defining Parameter':'Defining_Parameter',\
                                'index':'aq_unique_no',\
                                'Defining Site':'site_no'})
air_quality_df.head()

,aq_unique_no,State_Name,County,Date,AQI,Category,Defining_Parameter,site_no
0,0,California,Alameda,2010-01-01,72,Moderate,PM2.5,06-001-0007
1,1,California,Alameda,2010-01-02,51,Moderate,PM2.5,06-001-2004
2,2,California,Alameda,2010-01-03,60,Moderate,PM2.5,06-001-0007
3,3,California,Alameda,2010-01-04,90,Moderate,PM2.5,06-001-0007
4,4,California,Alameda,2010-01-05,90,Moderate,PM2.5,06-001-0011


In [4]:
# Defining parameter unique list

parameter = air_quality_df.groupby('Defining_Parameter')
parameter_unique = parameter['Defining_Parameter'].first()


defining_parameter_df = pd.DataFrame({'Parameter':parameter_unique}).reset_index().drop(columns=['Parameter'])

defining_parameter_df

,Defining_Parameter
0,CO
1,NO2
2,Ozone
3,PM10
4,PM2.5
5,SO2


In [5]:
# Sites SQL table prep
sites_table_df = sites_df.reset_index().rename(columns={'Defining Site':'site_no','Land Use':'Land_Use', \
                                'Location Setting':'Location_Setting', \
                                'State Name':'State_Name', \
                                'County Code':'County_Code', \
                                'City Name':'City_Name', \
                                'CBSA Name':'CBSA_Name'})\
                    .drop(columns=['County Name'])
                    

# site_count = sites_df.groupby('site_no')
# site_unique = site_count['County Name'].unique().count()
# print(site_unique)
sites_table_df.head()

,site_no,Latitude,Longitude,Elevation,Land_Use,Location_Setting,State_Name,County_Code,City_Name,CBSA_Name
0,06-001-0001,37.876870,-122.266913,15.0,RESIDENTIAL,URBAN AND CENTER CITY,California,1,Berkeley,"San Francisco-Oakland-Hayward, CA"
1,06-001-0002,37.533243,-121.958813,22.0,COMMERCIAL,URBAN AND CENTER CITY,California,1,Fremont,"San Francisco-Oakland-Hayward, CA"
2,06-001-0003,37.684900,-121.765900,150.0,COMMERCIAL,URBAN AND CENTER CITY,California,1,Livermore,"San Francisco-Oakland-Hayward, CA"
3,06-001-0004,37.800484,-122.266358,38.0,COMMERCIAL,URBAN AND CENTER CITY,California,1,Oakland,"San Francisco-Oakland-Hayward, CA"
4,06-001-0005,37.798600,-122.268400,10.0,COMMERCIAL,URBAN AND CENTER CITY,California,1,Oakland,"San Francisco-Oakland-Hayward, CA"


In [6]:
# Census Population SQL table prep
census_df = census_df.dropna().apply(lambda x: x.str.strip() if x.dtype == "object" else x)
census_count = census_df.dropna().groupby('County')

census_unique = census_count['County'].unique().count()
print(census_unique)
census_df.head()

58


,County,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,Alameda,1510271,1510258,1512986,1530915,1553764,1579593,1607792,1634538,1650950,1660196,1666756,1671329
2,Alpine,1175,1175,1161,1093,1110,1128,1080,1077,1047,1111,1089,1129
3,Amador,38091,38091,37886,37543,37104,36620,36726,37031,37429,38529,39405,39752
4,Butte,220000,220005,219949,219975,220869,221641,223516,224631,226231,228696,230339,219186
5,Calaveras,45578,45578,45468,45160,44815,44655,44671,44965,45322,45681,45698,45905


In [7]:
# County SQL table prep
county_name = sites_df.groupby('County Code')
county_name_unique = county_name['County Name'].first()
county_code_unique = county_name['County Name'].unique().count()

county_df = pd.DataFrame({'County':county_name_unique}).rename_axis('county_code')
county_table_df = county_df.reset_index().rename(columns={'County':'county_name'})
county_table_df.head()

,county_code,county_name
0,1,Alameda
1,3,Alpine
2,5,Amador
3,7,Butte
4,9,Calaveras


In [8]:
county_code_census = county_df.reset_index().merge(census_df,on='County')
county_code_census.head()

,county_code,County,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1,Alameda,1510271,1510258,1512986,1530915,1553764,1579593,1607792,1634538,1650950,1660196,1666756,1671329
1,3,Alpine,1175,1175,1161,1093,1110,1128,1080,1077,1047,1111,1089,1129
2,5,Amador,38091,38091,37886,37543,37104,36620,36726,37031,37429,38529,39405,39752
3,7,Butte,220000,220005,219949,219975,220869,221641,223516,224631,226231,228696,230339,219186
4,9,Calaveras,45578,45578,45468,45160,44815,44655,44671,44965,45322,45681,45698,45905


In [9]:
air_quality_table_df = county_df.reset_index().merge(air_quality_df,on='County').drop(columns=['County'])
air_quality_table_df.head()

,county_code,aq_unique_no,State_Name,Date,AQI,Category,Defining_Parameter,site_no
0,1,0,California,2010-01-01,72,Moderate,PM2.5,06-001-0007
1,1,1,California,2010-01-02,51,Moderate,PM2.5,06-001-2004
2,1,2,California,2010-01-03,60,Moderate,PM2.5,06-001-0007
3,1,3,California,2010-01-04,90,Moderate,PM2.5,06-001-0007
4,1,4,California,2010-01-05,90,Moderate,PM2.5,06-001-0011


In [10]:
# DateYear SQL table prep
unique_date = air_quality_table_df.groupby('Date')
date_unique = unique_date['Date'].first()


date_unique_df = pd.DataFrame({'Dates':date_unique}).reset_index().drop(columns=['Dates'])
date_unique_df['year'] = pd.DatetimeIndex(date_unique_df['Date']).year

date_unique_df

,Date,year
0,2010-01-01,2010
1,2010-01-02,2010
2,2010-01-03,2010
3,2010-01-04,2010
4,2010-01-05,2010
...,...,...
3952,2020-10-27,2020
3953,2020-10-28,2020
3954,2020-10-29,2020
3955,2020-10-30,2020


In [11]:
#defining year table
year = date_unique_df.groupby('year')
year_unique = year['year'].first()


year_df = pd.DataFrame({'years':year_unique}).reset_index().drop(columns=['years'])

year_df.head()

,year
0,2010
1,2011
2,2012
3,2013
4,2014


In [12]:
# prep census SQL table
county_code_census = county_code_census.drop(columns=['County', 'Census','Estimates Base']).set_index('county_code')
county_code_census.head()

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
county_code,,,,,,,,,,
1,1512986,1530915,1553764,1579593,1607792,1634538,1650950,1660196,1666756,1671329
3,1161,1093,1110,1128,1080,1077,1047,1111,1089,1129
5,37886,37543,37104,36620,36726,37031,37429,38529,39405,39752
7,219949,219975,220869,221641,223516,224631,226231,228696,230339,219186
9,45468,45160,44815,44655,44671,44965,45322,45681,45698,45905


In [13]:
county_code_census_pop_year = county_code_census.stack().reset_index()\
                                                .rename(columns={'level_1':'year',0:'population'})\
                                                .rename_axis('census_unique_no').reset_index()
county_code_census_pop_year

,census_unique_no,county_code,year,population
0,0,1,2010,1512986
1,1,1,2011,1530915
2,2,1,2012,1553764
3,3,1,2013,1579593
4,4,1,2014,1607792
...,...,...,...,...
575,575,115,2015,74045
576,576,115,2016,74952
577,577,115,2017,76578
578,578,115,2018,77557


In [14]:
engine = create_engine('sqlite:///AirQuality.db', echo=True)
sqlite_connection = engine.connect()

2021-03-15 11:35:05,532 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-15 11:35:05,534 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:05,536 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-15 11:35:05,537 INFO sqlalchemy.engine.base.Engine ()


In [15]:
# check table names in database
engine.table_names()

2021-03-15 11:35:07,046 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-03-15 11:35:07,048 INFO sqlalchemy.engine.base.Engine ()


[]

In [16]:
sqlite_table = "County"
county_table_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-03-15 11:35:09,853 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("County")
2021-03-15 11:35:09,854 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:09,857 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("County")
2021-03-15 11:35:09,858 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:09,861 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "County" (
	"index" BIGINT, 
	county_code BIGINT, 
	county_name TEXT
)


2021-03-15 11:35:09,862 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:09,872 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35:09,873 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_County_index" ON "County" ("index")
2021-03-15 11:35:09,875 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:09,885 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35:09,889 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-15 11:35:09,892 INFO sqlalchemy.engine.base.Engine INSERT INTO "County" ("index", county_code,

In [17]:
sqlite_table = "year"
year_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-03-15 11:35:10,877 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("year")
2021-03-15 11:35:10,879 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:10,880 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("year")
2021-03-15 11:35:10,881 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:10,883 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE year (
	"index" BIGINT, 
	year BIGINT
)


2021-03-15 11:35:10,884 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:10,894 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35:10,896 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_year_index ON year ("index")
2021-03-15 11:35:10,897 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:10,904 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35:10,907 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-15 11:35:10,908 INFO sqlalchemy.engine.base.Engine INSERT INTO year ("index", year) VALUES (?, ?)
2021-03-15 11:35:10,909 INFO sqlalchemy

In [18]:
sqlite_table = "DateYear"
date_unique_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-03-15 11:35:11,542 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("DateYear")
2021-03-15 11:35:11,543 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:11,545 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("DateYear")
2021-03-15 11:35:11,546 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:11,548 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "DateYear" (
	"index" BIGINT, 
	"Date" TEXT, 
	year BIGINT
)


2021-03-15 11:35:11,550 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:11,558 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35:11,559 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_DateYear_index" ON "DateYear" ("index")
2021-03-15 11:35:11,561 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:11,568 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35:11,571 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-15 11:35:11,595 INFO sqlalchemy.engine.base.Engine INSERT INTO "DateYear" ("index", "Date", year

In [19]:
sqlite_table = "Defining_Parameter"
defining_parameter_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-03-15 11:35:12,230 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Defining_Parameter")
2021-03-15 11:35:12,232 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:12,233 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Defining_Parameter")
2021-03-15 11:35:12,234 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:12,237 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Defining_Parameter" (
	"index" BIGINT, 
	"Defining_Parameter" TEXT
)


2021-03-15 11:35:12,238 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:12,247 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35:12,249 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_Defining_Parameter_index" ON "Defining_Parameter" ("index")
2021-03-15 11:35:12,251 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:12,261 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35:12,264 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-15 11:35:12,265 INFO sqlalchemy.engine.base.Eng

In [20]:
sqlite_table = "CensusPopulation"
county_code_census_pop_year.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-03-15 11:35:12,845 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("CensusPopulation")
2021-03-15 11:35:12,847 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:12,849 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("CensusPopulation")
2021-03-15 11:35:12,850 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:12,853 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "CensusPopulation" (
	"index" BIGINT, 
	census_unique_no BIGINT, 
	county_code BIGINT, 
	year TEXT, 
	population BIGINT
)


2021-03-15 11:35:12,855 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:12,863 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35:12,865 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_CensusPopulation_index" ON "CensusPopulation" ("index")
2021-03-15 11:35:12,866 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:12,874 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35:12,877 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-15 

In [21]:
sqlite_table = "Sites"
sites_table_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-03-15 11:35:16,105 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Sites")
2021-03-15 11:35:16,107 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:16,109 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Sites")
2021-03-15 11:35:16,110 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:16,114 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Sites" (
	"index" BIGINT, 
	site_no TEXT, 
	"Latitude" FLOAT, 
	"Longitude" FLOAT, 
	"Elevation" FLOAT, 
	"Land_Use" TEXT, 
	"Location_Setting" TEXT, 
	"State_Name" TEXT, 
	"County_Code" BIGINT, 
	"City_Name" TEXT, 
	"CBSA_Name" TEXT
)


2021-03-15 11:35:16,116 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:16,126 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35:16,127 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_Sites_index" ON "Sites" ("index")
2021-03-15 11:35:16,128 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:16,136 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35

In [22]:
sqlite_table = "AirQuality"
air_quality_table_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-03-15 11:35:16,535 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("AirQuality")
2021-03-15 11:35:16,535 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:16,536 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("AirQuality")
2021-03-15 11:35:16,537 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:16,539 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "AirQuality" (
	"index" BIGINT, 
	county_code BIGINT, 
	aq_unique_no BIGINT, 
	"State_Name" TEXT, 
	"Date" TEXT, 
	"AQI" BIGINT, 
	"Category" TEXT, 
	"Defining_Parameter" TEXT, 
	site_no TEXT
)


2021-03-15 11:35:16,541 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:16,549 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35:16,551 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_AirQuality_index" ON "AirQuality" ("index")
2021-03-15 11:35:16,552 INFO sqlalchemy.engine.base.Engine ()
2021-03-15 11:35:16,562 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-15 11:35:16,621 INFO sqlalchemy

In [23]:
sqlite_connection.close()

In [ ]:
# create connection to ETL_project_DB in postgres
engine = create_engine('postgresql://postgres:'+ password + '@localhost:5432/AirQuality_DB')
Postgresconnection = engine.connect()

In [ ]:
# load dataframes into postgres using pandas
county_table_df.to_sql(name='County', con=engine, if_exists='append', index=False)

year_df.to_sql(name='year', con=engine, if_exists='append', index=False)

date_unique_df.to_sql(name='DateYear', con=engine, if_exists='append', index=False)

defining_parameter_df.to_sql(name='Defining_Parameter', con=engine, if_exists='append', index=False)

county_code_census_pop_year.to_sql(name='CensusPopulation', con=engine, if_exists='append', index=False)

sites_table_df.to_sql(name='Sites', con=engine, if_exists='append', index=False)

air_quality_table_df.to_sql(name='AirQuality', con=engine, if_exists='append', index=False)